In [2]:
from modules.get_expected_points import get_expected_points
from data.raw.club_mapping import club_mapping
from modules.get_league_table import get_home_league_table
from modules.get_league_table import get_away_league_table
import pandas as pd
fixtures = pd.read_csv("data/processed/fixtures.csv")
fixtures = fixtures[fixtures["is_planned_tf"]==True]
expected_points = get_expected_points(fixtures)


In [4]:
league_table = get_away_league_table(2024, club_mapping)
league_table.to_csv("data/processed/away_league_table.csv", index=False)


In [5]:
table

,position,tm_id,points
0,1,62,36
1,2,941,29
2,3,377,27
3,4,197,26
4,5,1322,25
5,6,2311,20
6,7,5546,19
7,8,1897,19
8,9,13726,19
9,10,697,17
